In [ ]:
%matplotlib inline


Language Modeling with nn.Transformer and TorchText
===============================================================

Este es un tutorial sobre cómo entrenar un modelo de secuencia a secuencia que usa el nn.Transformer 


Define the model
----------------




En este tutorial, entrenamos un modelo ``nn.TransformerEncoder`` en un
tarea de modelado del lenguaje.  La tarea de modelado del lenguaje es asignar un
probabilidad de la probabilidad de una palabra dada (o una secuencia de palabras)
seguir una secuencia de palabras.  Una secuencia de tokens se pasa a la incrustación
capa primero, seguida de una capa de codificación posicional para dar cuenta del orden
de la palabra (ver el párrafo siguiente para más detalles).  los
``nn.TransformerEncoder`` consta de varias capas de
``nn.TransformerEncoderLayer``
Junto con la secuencia de entrada, se requiere una máscara de atención cuadrada porque la
las capas de autoatención en ``nn.TransformerEncoder`` solo pueden asistir
las primeras posiciones de la secuencia.  Para la tarea de modelado del lenguaje, cualquier
los tokens en las posiciones futuras deben estar enmascarados.  Para producir una probabilidad
distribución sobre palabras de salida, la salida de ``nn.TransformerEncoder``
El modelo se pasa a través de una capa lineal seguida de una función log-softmax.

In [ ]:
import math
from typing import Tuple

import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset

class TransformerModel(nn.Module):

    def __init__(self, ntoken: int, d_model: int, nhead: int, d_hid: int,
                 nlayers: int, dropout: float = 0.5):
        super().__init__()
        self.model_type = 'Transformer'# Definimos nuestro tipo de modelo
        self.pos_encoder = PositionalEncoding(d_model, dropout)# Iniciamos la clase "PositionalEncoding" en "pos_encoder"
        encoder_layers = TransformerEncoderLayer(d_model, nhead, d_hid, dropout)# Creamos una red transformer de autoatención y alimentación directa 
        self.transformer_encoder = TransformerEncoder(encoder_layers, nlayers)# Apilamos nuestras capas "encoder_layers"
        self.encoder = nn.Embedding(ntoken, d_model)# hacemos una tabla de búsqueda simple que almacena incrustaciones de un diccionario y tamaño fijos. 
        self.d_model = d_model# El número de características esperadas en la entrada
        self.decoder = nn.Linear(d_model, ntoken)# Capa de coneccion completa

        self.init_weights()

    # Iniciamos nuestros pesos de "encoder" con un rago de 0.1 ha -0.1
    def init_weights(self) -> None:
        initrange = 0.1
        self.encoder.weight.data.uniform_(-initrange, initrange)
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def forward(self, src: Tensor, src_mask: Tensor) -> Tensor:
        """
        Args:
            src: Tensor, shape [seq_len, batch_size]
            src_mask: Tensor, shape [seq_len, seq_len]

        Returns:
            output Tensor of shape [seq_len, batch_size, ntoken]
        """
        # Codificamos nuestros tonsor de datos
        src = self.encoder(src) * math.sqrt(self.d_model)
        # Añadimos nuestra informacion posicional
        src = self.pos_encoder(src)
        # Pasamos nuestro Tensor de datos a nuestra red (Transformers)
        output = self.transformer_encoder(src, src_mask)
        # Pasamos nuestro Tensor de datos a nuestras capa de coneccion completa
        output = self.decoder(output)
        return output


def generate_square_subsequent_mask(sz: int) -> Tensor:
    """Genera una matriz triangular superior de -inf, con ceros en diag."""
    return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)

El módulo ``PositionalEncoding`` inyecta información sobre el
posición relativa o absoluta de las fichas en la secuencia.  los
las codificaciones posicionales tienen la misma dimensión que las incrustaciones, de modo que
los dos se pueden resumir.  Aquí, usamos las funciones ``seno`` y ``coseno`` de
diferentes frecuencias

In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        # Iniciamos anuestra capa de "dropout"
        self.dropout = nn.Dropout(p=dropout)
        
        #Buscamos la posicion actual
        position = torch.arange(max_len).unsqueeze(1)
        #Busacamos la exponencial de los elementos para obtener el termino divisor
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        #Creamos informacion posicional binaria usando senos y cosenos
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        # Agregamos nuestra informacion posicional binaria
        x = x + self.pe[:x.size(0)]
        # Retornamos nuestra infromacion pasada por una capa de abandono
        return self.dropout(x)

Load and batch data
-------------------




Este tutorial usa ``torchtext`` para generar un conjunto de datos Wikitext-2.
 Para acceder a los conjuntos de datos de torchtext, instale torchdata siguiendo las instrucciones en https://github.com/pytorch/data.

divide the alphabet into 4 sequences of length 6:

El objeto de vocabulario se construye en base al conjunto de datos de entrenamiento y se usa para numerizar
fichas en tensores. Wikitext-2 representa tokens raros como `<unk>`.

Dado un vector 1-D de datos secuenciales, ``batchify()`` organiza los datos
en columnas ``batch_size``. Si los datos no se dividen uniformemente en
columnas ``batch_size``, luego los datos se recortan para que quepan. Por ejemplo, con
el alfabeto como los datos (longitud total de 26) y ``batch_size=4``, lo haríamos
dividir el alfabeto en 4 secuencias de longitud 6:


\begin{align}\begin{bmatrix}
  \text{A} & \text{B} & \text{C} & \ldots & \text{X} & \text{Y} & \text{Z}
  \end{bmatrix}
  \Rightarrow
  \begin{bmatrix}
  \begin{bmatrix}\text{A} \\ \text{B} \\ \text{C} \\ \text{D} \\ \text{E} \\ \text{F}\end{bmatrix} &
  \begin{bmatrix}\text{G} \\ \text{H} \\ \text{I} \\ \text{J} \\ \text{K} \\ \text{L}\end{bmatrix} &
  \begin{bmatrix}\text{M} \\ \text{N} \\ \text{O} \\ \text{P} \\ \text{Q} \\ \text{R}\end{bmatrix} &
  \begin{bmatrix}\text{S} \\ \text{T} \\ \text{U} \\ \text{V} \\ \text{W} \\ \text{X}\end{bmatrix}
  \end{bmatrix}\end{align}

Batching enables more parallelizable processing. However, batching means that
the model treats each column independently; for example, the dependence of
``G`` and ``F`` can not be learned in the example above.

In [ ]:
#!pip install torchdata

In [ ]:
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

# Dividimos nuestros datos de entrenamiento
train_iter = WikiText2(split='train')

# Creamos un tokenizador, ejemplo: https://pytorch.org/text/stable/data_utils.html
tokenizer = get_tokenizer('basic_english')
 
# Creamos nuestro vocabulario usando como iterable nuestros datos "train_iter" tokenizados(con nuestra funcion "map")
# y una lista de fichas especiales usamos el specials=['<unk>']
vocab = build_vocab_from_iterator(map(tokenizer, train_iter), specials=['<unk>'])
vocab.set_default_index(vocab['<unk>']) 

def data_process(raw_text_iter: dataset.IterableDataset) -> Tensor:
    """Convierte texto sin procesar en un tensor plano."""
    data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in raw_text_iter]
    return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

# train_iter fue "consumido" por el proceso de construcción del vocabulario,
# así que tenemos que crearlo de nuevo
train_iter, val_iter, test_iter = WikiText2()
train_data = data_process(train_iter)
val_data = data_process(val_iter)
test_data = data_process(test_iter)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def batchify(data: Tensor, bsz: int) -> Tensor:
    """Divide los datos en secuencias bsz separadas, eliminando elementos adicionales
    eso no encajaría limpiamente.

    Args:
        data: Tensor, shape [N]
        bsz: int, batch size

    Returns:
        Tensor of shape [N // bsz, bsz]
    """
    # Divimos el primer elemnto de nuestro tensor entre el tamaño de nuestro minilote y lo redondeamos hacia abajo
    seq_len = data.size(0) // bsz
    # Divimos nuestros datos entre la logitud de nuestros mini lotes (seq_len * bsz)
    data = data[:seq_len * bsz]
    # Cambiamos la forma de nuestro Tensor para que:
    # - Tenga la cantidad de columnas establecidas en "bsz"
    # - Tenga la cantidad de filas establecidas en "seq_len"
    # - El ".t()" devuelven los tal cual tensores 0-D y 1-D 
    # - El ".contiguous()" devuelve un tensor contiguo en memoria que contiene los mismos datos que selftensor
    #   (Esto es nesesario si no sabe, si la nueva forma de tendor coincide con los datos que tiene)
    data = data.view(bsz, seq_len).t().contiguous()

    # Movemos nuestros datos a nuestro dispositivo (CPU/GPU)
    return data.to(device)

# Definimos el tamaño de nuestro mini lote de entrenamiento
batch_size = 20
# Definimos el tamaño de nuestro mini lote de evalucion 
eval_batch_size = 10

# Creamos nuestros datos de entrenamiento
train_data = batchify(train_data, batch_size)  # shape [seq_len, batch_size]
# Creamos nuestros datos de Evalucion
val_data = batchify(val_data, eval_batch_size)
# Creamos nuestros datos de Testeo
test_data = batchify(test_data, eval_batch_size)

Funciones para generar entrada y secuencia de destino
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~




``get_batch()`` generates a pair of input-target sequences for
the transformer model. It subdivides the source data into chunks of
length ``bptt``. For the language modeling task, the model needs the
following words as ``Target``. For example, with a ``bptt`` value of 2,
we’d get the following two Variables for ``i`` = 0:



``get_batch()`` genera un par de secuencias de entrada-objetivo para
el modelo del transformador Subdivide los datos de origen en fragmentos de
longitud ``bptt``. Para la tarea de modelado del lenguaje, el modelo necesita la
siguientes palabras como ``Objetivo``. Por ejemplo, con un valor ``bptt`` de 2,
obtendríamos las siguientes dos Variables para ``i`` = 0:

Cabe señalar que los trozos están a lo largo de la dimensión 0, consistentes 
con la dimensión ``S`` en el modelo de Transformador.  La dimensión del lote
``N`` está a lo largo de la dimensión 1.




In [ ]:
bptt = 35
def get_batch(source: Tensor, i: int) -> Tuple[Tensor, Tensor]:
    """
    Args:
        source: Tensor, shape [full_seq_len, batch_size]
        i: int

    Returns:
        tuple (data, target), where data has shape [seq_len, batch_size] and
        target has shape [seq_len * batch_size]
    """
    seq_len = min(bptt, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].reshape(-1)
    return data, target

Iniciar una instancia
--------------------




Los hiperparámetros del modelo se definen a continuación.  El tamaño del vocabulario es
igual a la longitud del objeto de vocabulario.

In [ ]:
ntokens = len(vocab)# tamaño del vocabulario
emsize = 200        # dimensión de incrustación/embedding 
d_hid = 200         # dimensión del modelo de red feedforward en nn.TransformerEncoder
nlayers = 2         # número de nn.TransformerEncoderLayer en nn.TransformerEncoder
nhead = 2           # número de cabezas en nn.MultiheadAttention
dropout = 0.2       # probabilidad de abandono
model = TransformerModel(ntokens, emsize, nhead, d_hid, nlayers, dropout).to(device)

Run the model
-------------




Usamos `CrossEntropyLoss`con `SGD`(descenso de gradiente estocástico) optimizador. 
La tasa de aprendizaje se establece inicialmente en 5.0 y sigue un `StepLR`calendario. 
Durante el entrenamiento, usamos `nn.utils.clip_grad_norm\`
para evitar que los gradientes exploten.

In [ ]:
import copy
import time

criterion = nn.CrossEntropyLoss() # Criterio de evalucion
lr = 5.0  # ratio de aprendizaje/learning rate
optimizer = torch.optim.SGD(model.parameters(), lr=lr)# Optimizador
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95) #Decae la tasa de aprendizaje de cada grupo de parámetros por gamma cada épocas.

def train(model: nn.Module) -> None:
    model.train()  # activar el modo de entrenamiento
    total_loss = 0. # Iniciamos la perdida total
    log_interval = 200 # Definimos la logitud de los intervalos
    start_time = time.time()# iniciamos el tiempo actual
    src_mask = generate_square_subsequent_mask(bptt).to(device)#Genera una matriz triangular y la mueve al dispositivo(CPU/GPU)

    num_batches = len(train_data) // bptt # Definimos el numero de mini lotes
    for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
        data, targets = get_batch(train_data, i)# Obtenemos nuestras Caracteristicas/Features y nuestro Objetivo/Target
        batch_size = data.size(0)# Definimos el tamaño de nuestro mini lote
        if batch_size != bptt:  # solo en el último lote
            src_mask = src_mask[:batch_size, :batch_size]
        output = model(data, src_mask)# Obtenemos la salida de nuestro modelo
        loss = criterion(output.view(-1, ntokens), targets)# Calculamos la perdida del modelo

        optimizer.zero_grad()# Vaciamos los gradientes
        loss.backward()# Propagamos el error por la red
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)#Recorta la norma de gradiente de un iterable de parámetros.
        optimizer.step()# Optimizamos nuestro modelo

        total_loss += loss.item()# Sumamos nuestra perdida a la perdida total
        if batch % log_interval == 0 and batch > 0:
            lr = scheduler.get_last_lr()[0]# Obtenemos el ratio de aprendizaje/learning rate 
            ms_per_batch = (time.time() - start_time) * 1000 / log_interval# Obtenemos el tiempo de ejecucion 
            cur_loss = total_loss / log_interval# Obtenemos la perdida 
            ppl = math.exp(cur_loss)# Calculamos la Perplejidad del modelo
            print(f'| epoch {epoch:3d} | {batch:5d}/{num_batches:5d} batches | '
                  f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | '
                  f'loss {cur_loss:5.2f} | ppl {ppl:8.2f}')
            # Reiniciamos la perdida y el tiempo
            total_loss = 0
            start_time = time.time()

def evaluate(model: nn.Module, eval_data: Tensor) -> float:
    model.eval()  # activar el modo de evaluación
    total_loss = 0. # Iniciamos la perdida total
    src_mask = generate_square_subsequent_mask(bptt).to(device)#Genera una matriz triangular y la mueve al dispositivo(CPU/GPU)
    with torch.no_grad(): # Le decimos al modelo que NO aprenda de aqui
        for i in range(0, eval_data.size(0) - 1, bptt):
            data, targets = get_batch(eval_data, i) # Obtenemos nuestras Caracteristicas/Features y nuestro Objetivo/Target
            batch_size = data.size(0)
            if batch_size != bptt:# solo en el último lote
                src_mask = src_mask[:batch_size, :batch_size]
            output = model(data, src_mask)# Obtenemos la salida/prediccion de nuestro modelo
            output_flat = output.view(-1, ntokens)# Cambiamos la forma de nuestra salida/prediccion
            total_loss += batch_size * criterion(output_flat, targets).item()# Sumamos la perdida de nuestro modelo en esta prediccion a la perdida total
    return total_loss / (len(eval_data) - 1)

Recorre las épocas.  Guarde el modelo si la pérdida de validación es la mejor
hemos visto hasta ahora.  Ajuste la tasa de aprendizaje después de cada época.

In [ ]:
best_val_loss = float('inf') # Definimos nuestro mejor valor de perdida
epochs = 3 # Definimos el nuemero de epocas
best_model = None # Definimos nuestro mejor modelo

#Iteramos el modelo por cada epoca
for epoch in range(1, epochs + 1):
    epoch_start_time = time.time()# Contamos nuestro tiempo de inicio
    train(model)#Entrenamos nuestro modelo
    val_loss = evaluate(model, val_data)#Evaluamos nuestro modelo
    val_ppl = math.exp(val_loss)# Camculamos la tasa de Perplejidad
    elapsed = time.time() - epoch_start_time# Contamos nuestro tiempo transcurrido
    print('-' * 89)
    print(f'| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | '
          f'valid loss {val_loss:5.2f} | valid ppl {val_ppl:8.2f}')
    print('-' * 89)

    #Si la perdida de nuestro modelo es mejor que el anterior
    if val_loss < best_val_loss:
        best_val_loss = val_loss# Guardamos la nueva mejor perdida
        best_model = copy.deepcopy(model)# Guardamos nuetro modelo 

    scheduler.step()# Modificamos nuestre ratio de aprendizaje/learning rate

| epoch   1 |   200/ 2928 batches | lr 5.00 | ms/batch 26.43 | loss  8.13 | ppl  3384.21
| epoch   1 |   400/ 2928 batches | lr 5.00 | ms/batch 14.01 | loss  6.86 | ppl   953.98
| epoch   1 |   600/ 2928 batches | lr 5.00 | ms/batch 14.03 | loss  6.42 | ppl   613.99
| epoch   1 |   800/ 2928 batches | lr 5.00 | ms/batch 14.05 | loss  6.29 | ppl   540.28
| epoch   1 |  1000/ 2928 batches | lr 5.00 | ms/batch 14.06 | loss  6.17 | ppl   479.99
| epoch   1 |  1200/ 2928 batches | lr 5.00 | ms/batch 14.08 | loss  6.15 | ppl   469.37
| epoch   1 |  1400/ 2928 batches | lr 5.00 | ms/batch 14.06 | loss  6.10 | ppl   447.05
| epoch   1 |  1600/ 2928 batches | lr 5.00 | ms/batch 14.07 | loss  6.10 | ppl   446.20
| epoch   1 |  1800/ 2928 batches | lr 5.00 | ms/batch 15.12 | loss  6.01 | ppl   409.20
| epoch   1 |  2000/ 2928 batches | lr 5.00 | ms/batch 14.13 | loss  6.01 | ppl   406.01
| epoch   1 |  2200/ 2928 batches | lr 5.00 | ms/batch 14.14 | loss  5.89 | ppl   359.71
| epoch   1 |  2400/ 

Evaluar el mejor modelo en el conjunto de datos de prueba
-------------------------------------------




In [ ]:
test_loss = evaluate(best_model, test_data)
test_ppl = math.exp(test_loss)
print('=' * 89)
print(f'| End of training | test loss {test_loss:5.2f} | '
      f'test ppl {test_ppl:8.2f}')
print('=' * 89)

| End of training | test loss  5.51 | test ppl   247.52
